# Reduced Data Structures
This notebook chooses a small consistent subset of the treated dataset to be run in small machines. The generated dataset is limited to a number of **drug-drug interactions** specified by the variable $N$. From this dataset, it generates the corresponding data structures like dictionaries and adjacency matrices to be fed directly to DECAGON.<br>
This code is in part the adaptation in `pandas` of the script `drug_dataset.sh`, merged with `data_structures.ipynb`.

Author: Juan Sebastian Diaz Boada, May 2020

## Python 3

In [1]:
import numpy as np
import pandas as pd
import scipy.sparse as sp
import pickle
from joblib import Parallel, delayed

In [2]:
# Number of side effects 
#(Maximum is 1317, but the max chosen is 964 to be consistents with authors of decagon)
N = 964
RED = True

### Import databases

In [3]:
PPI = pd.read_csv('affinities/small-decagon-ppi.csv',sep=',')
AF = pd.read_csv('affinities/small-decagon-affinities.csv',sep=',')
DTI = pd.read_csv('affinities/small-decagon-targets.csv',sep=',')
DDI = pd.read_csv('affinities/small-decagon-combo.csv',sep=',')
DSE = pd.read_csv('affinities/small-decagon-mono.csv',sep=',')
SE = pd.read_csv('original_data/bio-decagon-effectcategories.csv',sep=',')

In [4]:
# Number of interactions
orig_ppi = len(PPI.index)
orig_af = len(AF.index)
orig_dti = len(DTI.index)
orig_ddi = len(DDI.index)
orig_dse = len(DSE.index)
# Number of nodes
orig_ddi_drugs = len(pd.unique(DDI[['STITCH 1','STITCH 2']].values.ravel()))
orig_ppi_genes = len(pd.unique(PPI[['Gene 1','Gene 2']].values.ravel()))
orig_dti_drugs = len(pd.unique(DTI['STITCH']))
orig_dti_genes = len(pd.unique(DTI['Gene']))
orig_dse_drugs = len(pd.unique(DSE['STITCH'].values))
orig_af_genes = len(pd.unique(AF['Target'].values))
orig_af_drugs = len(pd.unique(AF['Drug'].values))
# Side effects
orig_se_mono = len(pd.unique(DSE['Side Effect Name']))
orig_se_combo = len(pd.unique(DDI['Polypharmacy Side Effect'].values))

### Choose Side effects

In [5]:
# Sort DDI to be consistent with the authors
DDI['freq'] = DDI.groupby('Polypharmacy Side Effect')['Polypharmacy Side Effect']\
            .transform('count')
DDI = DDI.sort_values(by=['freq'], ascending=False).drop(columns=['freq'])
se = pd.unique(DDI['Polypharmacy Side Effect'].values)
se_name = pd.unique(DDI['Side Effect Name'].values)

In [6]:
# Here we choose specific side effects: Diarrhea, Emesis, Increased body temp,
# bleeding, Renal disorder, Leucopenia and Icterus
# ignoring the previous given number N
if RED:
    names = ['diarrhea', 'emesis', 'body temperature increased', 'bleeding', 'disorder renal',\
             'leucopenia','icterus']
    idx = []
    for i in range(len(se_name)):
        if se_name[i].lower() in names:
            print(i,se_name[i])
            idx.append(i)
    #idx = list(range(len(names)))
    print(idx)
    se = se[idx]
    N = len(se);
    print('N=',N)
else:
    se = se[:N]
    print('N=',len(se))

8 diarrhea
10 emesis
24 body temperature increased
86 Bleeding
156 disorder Renal
245 leucopenia
248 icterus
[8, 10, 24, 86, 156, 245, 248]
N= 7


### Select DDIs

In [7]:
DDI = DDI[DDI['Polypharmacy Side Effect'].isin(se)].reset_index(drop=True)
DDI_drugs = pd.unique(DDI[['STITCH 1','STITCH 2']].values.ravel()) # Unique drugs 
drug2idx = {drug: i for i, drug in enumerate(DDI_drugs)}
se_names = pd.unique(DDI['Side Effect Name']) # Unique joint side effects
se_combo_name2idx = {se: i for i, se in enumerate(se_names)}
n_drugs = len(DDI_drugs)

### Select Drug side effects

In [8]:
DSE = DSE[DSE['STITCH'].isin(DDI_drugs)].reset_index(drop=True)
dse_drugs = len(pd.unique(DSE['STITCH'].values))
se_mono_names = pd.unique(DSE['Side Effect Name'].values) # Unique individual side effects
se_mono_name2idx = {name: i for i, name in enumerate(se_mono_names)}
n_semono = len(se_mono_names)

### Select DTIs

In [9]:
DTI = DTI[DTI['STITCH'].isin(DDI_drugs)].reset_index(drop=True)
DTI_genes = pd.unique(DTI['Gene']) # Unique genes in DTI
DTI_drugs = pd.unique(DTI['STITCH']) # Unique drugs in DTI
dti_drugs = len(DTI_drugs)
dti_genes = len(DTI_genes)

### Select Affinity features

In [10]:
AF = AF[np.logical_and(AF['Target'].isin(DTI_genes),AF['Drug'].isin(DTI_drugs))]\
    .reset_index(drop=True)
af_genes = len(pd.unique(AF['Target'].values))
af_drugs = len(pd.unique(AF['Drug'].values))

### Select PPIs

In [11]:
PPI = PPI[np.logical_or(PPI['Gene 1'].isin(DTI_genes),
                       PPI['Gene 2'].isin(DTI_genes))].reset_index(drop=True)
PPI_genes = pd.unique(PPI[['Gene 1','Gene 2']].values.ravel()) # Unique genes is PPI
gene2idx = {gene: i for i, gene in enumerate(PPI_genes)}
n_genes = len(PPI_genes)

## Adjacency matrices and degrees

In [12]:
# DDI adjacency matrices
def se_adj_matrix(se_name):
    m = np.zeros([n_drugs,n_drugs],dtype=int)
    seDDI = DDI[DDI['Side Effect Name'].str.match(se_name)].reset_index()
    for j in seDDI.index:
        row = drug2idx[seDDI.loc[j,'STITCH 1']]
        col = drug2idx[seDDI.loc[j,'STITCH 2']]
        m[row,col] = m[col,row] = 1
    return sp.csr_matrix(m), se_name
ddi_adj_list,se_name= zip(*Parallel(n_jobs=8)\
    (delayed(se_adj_matrix)(d) for d in se_combo_name2idx.keys()))
ddi_degrees_list = [np.array(drug_adj.sum(axis=0)).squeeze() for drug_adj in ddi_adj_list]

In [13]:
print(se_name)
print(se_combo_name2idx)
print(n_genes)

('diarrhea', 'emesis', 'body temperature increased', 'Bleeding', 'disorder Renal', 'leucopenia', 'icterus')
{'diarrhea': 0, 'emesis': 1, 'body temperature increased': 2, 'Bleeding': 3, 'disorder Renal': 4, 'leucopenia': 5, 'icterus': 6}
16814


In [14]:
# DTI adjacency matrices (affinity and adjacency)
dti_affinity = np.zeros([n_genes,n_drugs])
for i in AF.index:
    gene = AF.loc[i,'Target']
    drug = AF.loc[i,'Drug']
    row = gene2idx[gene]
    col = drug2idx[drug]
    aff = AF[np.logical_and(AF['Target']==gene,AF['Drug']==drug)]['Affinity_norm'].values[0]
    dti_affinity[row,col] = aff
dti_affinity = sp.csr_matrix(dti_affinity)

In [15]:
dti_adj = np.zeros([n_genes,n_drugs],dtype=np.int8)
for i in DTI.index:
    gene = DTI.loc[i,'Gene']
    drug = DTI.loc[i,'STITCH']
    row = gene2idx[gene]
    col = drug2idx[drug]
    dti_adj[row,col] = 1
dti_adj = sp.csr_matrix(dti_adj)

In [17]:
print(dti_affinity.nonzero()[0].shape)
print(dti_adj.nonzero()[0].shape)

(17142,)
(17151,)


In [18]:
# PPI adjacency matrix and degrees
ppi_adj = np.zeros([n_genes,n_genes],dtype=int)
for i in PPI.index:
    row = gene2idx[PPI.loc[i,'Gene 1']]
    col = gene2idx[PPI.loc[i,'Gene 2']]
    ppi_adj[row,col]=ppi_adj[col,row]=1
ppi_degrees = np.sum(ppi_adj,axis=0)
ppi_adj = sp.csr_matrix(ppi_adj)

In [19]:
# Drug Feature matrix
drug_feat = np.zeros([n_drugs,n_semono],dtype=int)
for i in DSE.index:
    row = drug2idx[DSE.loc[i,'STITCH']]
    col = se_mono_name2idx[DSE.loc[i,'Side Effect Name']]
    drug_feat[row,col] = 1
drug_feat = sp.csr_matrix(drug_feat)

### Print

In [20]:
# Interactions (edges)
print('Interactions (edges)')
print('Original number of PPI interactions:', orig_ppi)
print('New number of PPI interactions:', len(PPI.index))
print('\n')
print('Original number of DTI interactions:', orig_dti)
print('New number of DTI interactions:', len(DTI.index))
print('\n')
print('Original number of DDI interactions:', orig_ddi)
print('New number of DDI interactions:', len(DDI.index))
print('\n')
print('Original number of DSE interactions:', orig_dse)
print('New number of DSE interactions:', len(DSE.index))
print('\n')
print('Original number of AF interactions:', orig_af)
print('New number of AF interactions:', len(AF.index))
print('\n')
# Drugs and genes (nodes)
print('Drugs and genes (nodes)')
print("Original number of drugs in DSE:",orig_dse_drugs)
print("New number of drugs in DSE:",dse_drugs)
print('\n')
print("Original number drugs in DTI",orig_dti_drugs)
print("New number of drugs in DTI",dti_drugs)
print('\n')
print('Original number of genes in DTI:', orig_dti_genes)
print('New number of genes in DTI:',dti_genes)
print('\n')
print('Original number of genes:',orig_ppi_genes)
print('New number of genes:', n_genes)
print('\n')
print('Original number of drugs:',orig_ddi_drugs)
print('New number of drugs:', n_drugs)
print('\n')
print("Original number of genes in AF:",orig_af_genes)
print("New number of genes in AF:",af_genes)
print('\n')
print("Original number of drugs in AF:",orig_af_drugs)
print("New number of drugs in AF:",af_drugs)
print('\n')
# Side effects
print('Side effects')
print('Original number of joint side effects:',orig_se_combo)
print('New number of joint side effects:', len(se_names))
print('\n')
print('Original number of single side effects:', orig_se_mono)
print('New number of single side effects:', n_semono)

Interactions (edges)
Original number of PPI interactions: 585448
New number of PPI interactions: 318062


Original number of DTI interactions: 17802
New number of DTI interactions: 17151


Original number of DDI interactions: 1208035
New number of DDI interactions: 24390


Original number of DSE interactions: 81275
New number of DSE interactions: 80716


Original number of AF interactions: 17802
New number of AF interactions: 17151


Drugs and genes (nodes)
Original number of drugs in DSE: 282
New number of drugs in DSE: 276


Original number drugs in DTI 280
New number of drugs in DTI 274


Original number of genes in DTI: 3534
New number of genes in DTI: 3534


Original number of genes: 18240
New number of genes: 16814


Original number of drugs: 282
New number of drugs: 276


Original number of genes in AF: 3534
New number of genes in AF: 3534


Original number of drugs in AF: 280
New number of drugs in AF: 274


Side effects
Original number of joint side effects: 1308
New number of

## Save

In [21]:
data = {}
# Dictionaries
data['gene2idx'] = gene2idx
data['drug2idx'] = drug2idx
data['se_mono_name2idx'] = se_mono_name2idx
data['se_combo_name2idx'] = se_combo_name2idx
# DDI
data['ddi_adj_list'] = ddi_adj_list
data['ddi_degrees_list'] = ddi_degrees_list
# DTI (&AF)
data['dti_affinity'] = dti_affinity
data['dti_adj'] = dti_adj
# PPI
data['ppi_adj'] = ppi_adj
data['ppi_degrees'] = ppi_degrees
# DSE
data['drug_feat'] = drug_feat

In [23]:
filename = './data_structures/DS/DS_real_affinity_DSE_' + str(n_semono) +'_genes_' +\
str(n_genes) + '_drugs_' + str(n_drugs) + '_se_' + str(N)
print(filename)

./data_structures/DS/DS_real_affinity_DSE_8120_genes_16814_drugs_276_se_7


In [24]:
with open(filename, 'wb') as f:
    pickle.dump(data, f, protocol=3)